In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from experiment import *

# Load data ** Chose your path before get stated **

In [2]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS1_5classes__Fourier.csv'
dataset = pd.read_csv(os.path.join(rootDir, path, filename));

In [3]:
dataset.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,target
0,0.140163,0.302116,0.256512,0.199795,0.167456,0.136428,0.092218,0.053257,0.035375,0.022090,...,0.001537,0.003202,0.000649,0.002842,0.002034,0.003303,0.002120,0.001724,0.001094,0
1,0.164694,0.294092,0.248227,0.189410,0.155726,0.119119,0.084469,0.051921,0.026894,0.022658,...,0.001776,0.001309,0.001864,0.002886,0.001600,0.001463,0.001042,0.000314,0.001563,0
2,0.264069,0.336712,0.279655,0.212017,0.169007,0.126995,0.081884,0.039922,0.025413,0.012168,...,0.002457,0.002108,0.001981,0.001673,0.002745,0.001638,0.001236,0.002879,0.002812,0
3,0.072350,0.351005,0.288275,0.218995,0.177219,0.124068,0.074372,0.044149,0.024169,0.016180,...,0.001813,0.001525,0.002232,0.003359,0.001211,0.002223,0.001470,0.002496,0.000781,0
4,0.029850,0.354420,0.295535,0.229964,0.184878,0.129478,0.076174,0.041396,0.027271,0.015194,...,0.002030,0.000666,0.001806,0.002329,0.001273,0.002032,0.001337,0.002298,0.001719,0


# Data preprocessing

In this step we are going standardize our dataset.

In [4]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

Separete features from labels.

In [5]:
data_std = std_scaler.fit_transform(dataset.values[:,:-1])
data_label = dataset.values[:,-1]

# Load classifiers

In [6]:
# linear models:
from sklearn.externals import joblib

random_search = joblib.load('./Models/classifiers_MLP.pkl')

In [7]:
random_search

{'MLP': RandomizedSearchCV(cv=8, error_score='raise',
           estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(100,), learning_rate='adaptive',
        learning_rate_init=0.0005, max_iter=1300, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
        verbose=False, warm_start=False),
           fit_params=None, iid=True, n_iter=20, n_jobs=100,
           param_distributions={'hidden_layer_sizes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, ...81, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]},
     

Load the best estimation of hyperameters after randomly searching it.

In [8]:
mlp_clf = random_search['MLP'].best_estimator_

## Hyperameter tunning by randomized search:

Classifiers definitions:

In [9]:
from sklearn.naive_bayes import GaussianNB

naiveBayes_clf = GaussianNB()

Define our models to train. Import each classifiers into a python dictionary.

In [10]:
classifiers = {'MLP': mlp_clf, 'Naive-Bayes': naiveBayes_clf}

# Classifiers for training an testing

Stratified cross-validation for model selection are going to be used.

In [11]:
kfolds = 8
clf_outputs = run_classifiers(data_std, data_label, classifiers, kfolds)

# Performance assessment

Saving variables in a dictionary:

In [12]:
results = {}
results['train'] = results_clf(5, clf_outputs['train']['true'], clf_outputs['train']['pred'])
results['test'] = results_clf(5, clf_outputs['test']['true'], clf_outputs['test']['pred'])

/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/navar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/media/navar/LargeData/Navar/Dados_diversos/ECG-01-paper_01/ECG-Arritmia-Paper1/Results_Navar/Fourier/Experiment_03_scikitLearn__allCLF/experiment.py:152: RuntimeWarning: invalid value encountered in double_scalars
  metrics_class[i,3] = (2*metrics_class[i,2]*metrics_class[i,1])/(metrics_class[i,2] + metrics_class[i,1])


# Savel results, models and pipeline to a .pkl file

In [13]:
save_models(classifiers)
save_pipeline(std_scaler)

# Save results to CSVs and figures

In [14]:
export_results(results['test'], 'test')
export_results(results['train'], 'train')

MLP
Naive-Bayes
MLP
Naive-Bayes


In [15]:
results['train']['Naive-Bayes']

{'confMat': [array([[11740, 23894,  3104,   997,   362],
         [  143,   655,     8,    12,     5],
         [  840,   575,  1460,   174,   264],
         [   79,    68,    47,   166,     3],
         [    2,     1,     1,     0,     3]]),
  array([[12355, 23408,  2245,  1293,   796],
         [  164,   639,     6,     8,     6],
         [  977,   281,  1614,   242,   199],
         [   72,    82,    56,   149,     4],
         [    3,     0,     1,     0,     3]]),
  array([[ 7778, 26404,  3648,  1224,  1044],
         [   95,   676,     5,    17,    30],
         [ 1108,   293,  1582,   300,    30],
         [   59,   105,    49,   147,     3],
         [    1,     2,     1,     0,     3]]),
  array([[ 6646, 27898,  3990,   920,   644],
         [  121,   670,    10,    18,     4],
         [  441,   694,  1811,   147,   221],
         [   34,   121,    67,   137,     4],
         [    2,     1,     0,     0,     4]]),
  array([[ 8241, 27634,  2583,  1026,   614],
         [  148

StandardScaler(copy=True, with_mean=True, with_std=True)


In [7]:
dataset.head()

,Var1,Var2,Var3,Var4,Var5,Var9
0,0.834552,0.666667,2.576261,0.417694,0.923901,0
1,0.852697,0.673684,2.490621,0.421475,0.651352,0
2,0.848921,0.666667,2.537718,0.429076,0.708984,0
3,0.854177,0.688172,2.570455,0.390772,0.580959,0
4,0.858868,0.673684,2.497976,0.438510,0.630987,0
